In [1]:
import os
os.environ["AOC_SESSION"] = "53616c7465645f5fe3a26a621eccffe07198b16f4da73d3bd101febe1b9c6428f4d781679e113c58fa1dc25c83d043b5"

In [2]:
import pandas as pd
import numpy as np
from aocd.models import Puzzle
from aocd.transforms import numbers, lines

# Day 3

## Part 1

In [ ]:
from aocd.models import Puzzle
from aocd.transforms import numbers
puzzle = Puzzle(year=2021, day=3)
data = numbers(puzzle.input_data)

In [ ]:
data[:10]

In [ ]:
# How many bits does each line have?
max([len(str(i)) for i in data])

In [ ]:
data_str = [str(i).zfill(12) for i in data]
data_str[:10]

In [ ]:
data_str = [[b for b in s] for s in data_str]
data_str[:10]

In [ ]:
df = pd.DataFrame(data_str).astype(int)

gamma_rate = ''.join((df.sum(axis = 0) > 500).astype(int).astype(str))
epsilon_rate = ''.join((df.sum(axis = 0) < 500).astype(int).astype(str))

print(gamma_rate)
print(epsilon_rate)

In [ ]:
int(gamma_rate, 2)

In [ ]:
int(epsilon_rate, 2)

In [ ]:
int(gamma_rate, 2)*int(epsilon_rate, 2)

## Part 2

In [ ]:
current_result_oxygen = df.copy()

for i in range(12):
    
    counts = current_result_oxygen[i].value_counts()
    
    # If both digits are equally common
    if len(counts) == 2 and counts[0] == counts[1]:
        current_result_oxygen = current_result_oxygen[current_result_oxygen[i] == 1]
    else:
        most_common = counts.idxmax()
        current_result_oxygen = current_result_oxygen[current_result_oxygen[i] == most_common]
        
    if len(current_result_oxygen) == 1: break

In [ ]:
oxygen_generator_rating = ''.join(current_result_oxygen.astype(str).iloc[0])
print(oxygen_generator_rating)
print(int(oxygen_generator_rating, 2))

In [ ]:
current_result_co2 = df.copy()

for i in range(12):
    
    counts = current_result_co2[i].value_counts()

    # If both digits are equally common
    if len(counts) == 2 and counts[0] == counts[1]:
        current_result_co2 = current_result_co2[current_result_co2[i] == 0]
    else:
        least_common = counts.idxmin()
        current_result_co2 = current_result_co2[current_result_co2[i] == least_common]
        
    if len(current_result_co2) == 1: break

In [ ]:
co2_scrubber_rating = ''.join(current_result_co2.astype(str).iloc[0])
print(co2_scrubber_rating)
print(int(co2_scrubber_rating, 2))

In [ ]:
int(oxygen_generator_rating, 2)*int(co2_scrubber_rating, 2)

# Day 4

## Part 1

In [ ]:
puzzle = Puzzle(year=2021, day=4)
data = lines(puzzle.input_data)
data = [d for d in data if d != '']
data

In [ ]:
drawn_numbers = data[0].split(',')
boards = [[x for x in board.split(' ') if x] for board in data[1:]]
boards = np.array([boards[i*5:(i+1)*5] for i in range(100)])
boards

In [ ]:
def check_if_board_wins(board):
    return (board.sum(axis = 0) == 5).any() or (board.sum(axis = 1) == 5).any()

marked_boards = np.zeros(boards.shape).astype(bool)

for number in drawn_numbers:

    marked_boards = marked_boards | (boards == number)
    winning_boards = list(map(check_if_board_wins, marked_boards))
    
    if any(winning_boards):
        print("We have a winner!")
        last_called_number = int(number)
        print(f"Called number: {last_called_number}")
        winner = boards[winning_boards, :, :]
        print(winner)
        break

In [ ]:
marked_boards[winning_boards, :, :]

In [ ]:
winner_marked_numbers = marked_boards[winning_boards, :, :]
unmarked_numbers = winner.astype(int)*(1 - winner_marked_numbers)
unmarked_numbers = unmarked_numbers[0, :, :]
unmarked_numbers.sum()

In [ ]:
unmarked_numbers.sum()*last_called_number

## Part 2

In [ ]:
def check_if_board_does_not_win(board):
    return ((board.sum(axis = 0) == 5).sum() + (board.sum(axis = 1) == 5).sum()) == 0

marked_boards = np.ones(boards.shape).astype(bool)

for number in drawn_numbers[::-1]:
    
    marked_boards[(boards == number)] = False
    boards_on_the_verge_of_winning = list(map(check_if_board_does_not_win, marked_boards))
    
    if any(boards_on_the_verge_of_winning):
        print("We have the last winner!")
        last_called_number = int(number)
        print(f"Called number: {last_called_number}")
        winner = boards[boards_on_the_verge_of_winning, :, :]
        print(winner)
        break

In [ ]:
marked_boards[boards_on_the_verge_of_winning, :, :]

In [ ]:
winner_marked_numbers = marked_boards[boards_on_the_verge_of_winning, :, :]
unmarked_numbers = winner.astype(int)*(1 - winner_marked_numbers)
unmarked_numbers = unmarked_numbers[0, :, :]
unmarked_numbers.sum() - last_called_number

In [ ]:
(unmarked_numbers.sum() - last_called_number) * last_called_number

# Day 5

## Part 1

In [ ]:
puzzle = Puzzle(year=2021, day=5)
data = lines(puzzle.input_data)
data[:5]

In [ ]:
"""data = ['0,9 -> 5,9',
'8,0 -> 0,8',
'9,4 -> 3,4',
'2,2 -> 2,1',
'7,0 -> 7,4',
'6,4 -> 2,0',
'0,9 -> 2,9',
'3,4 -> 1,4',
'0,0 -> 8,8',
'5,5 -> 8,2']""";

In [ ]:
start_points = np.asarray([x.split(' -> ')[0].split(',') for x in data]).astype(int)
end_points = np.asarray([x.split(' -> ')[1].split(',') for x in data]).astype(int)

In [ ]:
floor_size = max(start_points.max(), end_points.max()) + 1
floor = np.zeros((floor_size, floor_size))

horizontal_lines_idx = start_points[:, 0] == end_points[:, 0]
vertical_lines_idx = start_points[:, 1] == end_points[:, 1]

# horizontal lines
for idx in np.where(start_points[:, 0] == end_points[:, 0])[0]:
    x1, y1 = start_points[idx, :]
    x2, y2 = end_points[idx, :]
    if y2 < y1:
        input_score_board, y1
        y1 = y2
        y2 = input_score_board, output_score_board
    floor[x1, y1:y2 + 1] += 1
    
# vertical lines
for idx in np.where(start_points[:, 1] == end_points[:, 1])[0]:
    x1, y1 = start_points[idx, :]
    x2, y2 = end_points[idx, :]
    if x2 < x1:
        input_score_board, x1
        x1 = x2
        x2 = input_score_board, output_score_board
    floor[x1:x2+1, y1] += 1

In [ ]:
(floor >= 2).sum()

## Part 2

In [ ]:
# Lines are diagonal is difference between x2 and x1 is equal to the difference between y2 and y1
floor_w_diagonal = floor.copy()

In [ ]:
floor_w_diagonal = floor.copy()
directions = end_points - start_points

for idx in np.where(abs(directions)[:, 0] == abs(directions)[:, 1])[0]:

    direction = end_points[idx] - start_points[idx]
    n_steps = abs(direction[0])

    for i in range(n_steps + 1):
        x, y = (start_points[idx] + direction*i/n_steps).astype(int)
        floor_w_diagonal[x, y] += 1

In [ ]:
(floor_w_diagonal >= 2).sum()

# Day 6

In [ ]:
puzzle = Puzzle(year=2021, day=6)
data = puzzle.input_data.split(',')
data = [int(d) for d in data]
data[:5]

In [ ]:
def count_n_fish(data, n_days):
    fish_level_count = pd.value_counts(data)
    fish_level_count = fish_level_count.reindex(index = list(range(9))).fillna(0).astype(int)
    for day in range(n_days):
        state_at_end_of_day = fish_level_count.shift(-1).fillna(0)
        state_at_end_of_day[8] += fish_level_count[0]
        state_at_end_of_day[6] += fish_level_count[0]
        fish_level_count = state_at_end_of_day.copy()
    return fish_level_count.sum()

In [ ]:
print(count_n_fish([3,4,3,1,2], 18))
print(count_n_fish([3,4,3,1,2], 80))
print(count_n_fish([3,4,3,1,2], 256))

In [ ]:
print(count_n_fish(data, 18))
print(count_n_fish(data, 80))
print(count_n_fish(data, 256))

In [ ]:
%%time
print(count_n_fish(data, int(1000)))

In [ ]:
%%time
print(count_n_fish(data, int(10000)))

# Day 7

In [ ]:
puzzle = Puzzle(year=2021, day=7)
data = puzzle.input_data.split(',')
data = np.asarray([int(d) for d in data])
data[:5]

In [ ]:
best_fuel = np.inf
best_position = 0

for final_position in range(data.min(), data.max() + 1):
    fuel_spent = abs(data - final_position).sum()
    if fuel_spent < best_fuel:
        best_fuel = fuel_spent
        best_position = final_position
print(best_position, best_fuel)

In [ ]:
best_fuel = np.inf
best_position = 0

for final_position in range(data.min(), data.max() + 1):
    
    fuel_spent = 0
    for crab_position in data:
        fuel_spent += sum(range(abs(crab_position - final_position) + 1))
    
    if fuel_spent < best_fuel:
        best_fuel = fuel_spent
        best_position = final_position
print(best_position, best_fuel)

# Day 8

## Part 1

In [ ]:
puzzle = Puzzle(year=2021, day=8)
data = lines(puzzle.input_data)#.split(',')
#data = np.asarray([int(d) for d in data])
data[:5]

In [ ]:
output_digits = [e.split(' | ')[1] for e in data]
output_digits_joined = ' '.join(output_digits).split(' ')
output_digits_joined[:5]

In [ ]:
output_length = list(map(len, output_digits_joined))

In [ ]:
(np.array(output_length) == 2).sum() + (np.array(output_length) == 3).sum() + (np.array(output_length) == 4).sum() + (np.array(output_length) == 7).sum()

## Part 2

In [ ]:
input_digits = [e.split(' | ')[0] for e in data]
def contains_n_character_string(s, n_characters):
    return n_characters in list(map(len, s.split(' ')))

In [ ]:
all(map(lambda x: contains_n_character_string(x, 2), input_digits))

In [ ]:
all(map(lambda x: contains_n_character_string(x, 3), input_digits))

In [ ]:
all(map(lambda x: contains_n_character_string(x, 4), input_digits))

In [ ]:
all(map(lambda x: contains_n_character_string(x, 7), input_digits))

In [ ]:
data[0]

In [ ]:
def decode_digits(line):
    
    input_pattern, output_pattern = line.split(' | ')
    
    def segment_to_digit_mapping(s, segment_mapping):
    
        if len(s) == 2: return 1
        if len(s) == 3: return 7
        if len(s) == 4: return 4
        if len(s) == 7: return 8

        scrambled_string = np.asarray([letter for letter in s])
        unscrambled_string = np.asarray([letter for letter in s])
        for original_key, scrambled_key in segment_mapping.items():
            unscrambled_string[scrambled_string == scrambled_key] = original_key
        s = ''.join(unscrambled_string)

        if 'd' not in s: return 0
        if 'f' not in s: return 2
        if 'e' in s: return 6
        if 'c' not in s: return 5
        if 'b' in s: return 9
        return 3
    
    segment_mapping = {}

    pattern = np.asarray(input_pattern.split(' '))
    input_digit_length = np.asarray(list(map(len, pattern)))
    digit_w_2_character = pattern[input_digit_length == 2][0]
    digit_w_3_character = pattern[input_digit_length == 3][0]
    digit_w_4_character = pattern[input_digit_length == 4][0]
    digit_w_7_character = pattern[input_digit_length == 7][0]

    segment_mapping['a'] = [letter for letter in digit_w_3_character if letter not in digit_w_2_character][0]

    possible_right_segments = [letter for letter in digit_w_2_character]
    possible_top_left_middle_segments = [letter for letter in digit_w_4_character if letter not in possible_right_segments]
    possible_bottom_left_bottom_segments = [letter for letter in digit_w_7_character if letter != segment_mapping['a'] and letter not in possible_right_segments and letter not in possible_top_left_middle_segments]

    digits_w_6_characters = pattern[input_digit_length == 6]
    for digit in digits_w_6_characters:
        missing_character = [letter for letter in ['a', 'b', 'c', 'd', 'e', 'f', 'g'] if letter not in digit][0]
        if missing_character in possible_right_segments:
            segment_mapping['c'] = missing_character
            possible_right_segments.remove(missing_character)
            segment_mapping['f'] = possible_right_segments[0]
        elif missing_character in possible_top_left_middle_segments:
            segment_mapping['d'] = missing_character
            possible_top_left_middle_segments.remove(missing_character)
            segment_mapping['b'] = possible_top_left_middle_segments[0]
        elif missing_character in possible_bottom_left_bottom_segments:
            segment_mapping['e'] = missing_character
            possible_bottom_left_bottom_segments.remove(missing_character)
            segment_mapping['g'] = possible_bottom_left_bottom_segments[0]

    return ''.join(map(lambda s: str(segment_to_digit_mapping(s, segment_mapping)), output_pattern.split(' ')))

In [ ]:
decoded_values = [int(x) for x in list(map(decode_digits, data))]
decoded_values

In [ ]:
sum(decoded_values)

# Day 9

In [ ]:
puzzle = Puzzle(year=2021, day=9)
data = lines(puzzle.input_data)
data = list(map(lambda x: [e for e in x], data))
data = np.asarray(data).astype(int)
data

In [ ]:
data = ['2199943210',
'3987894921',
'9856789892',
'8767896789',
'9899965678']
data = list(map(lambda x: [e for e in x], data))
data = np.asarray(data).astype(int)
data

In [ ]:
data.shape

In [ ]:
low_points = []
low_point_positions = []

for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        
        point = data[i, j]
        # Check if top is lower
        if i != 0 and data[i-1, j] <= point: continue
        # Check if bottom is lower
        if i != (data.shape[0] - 1) and data[i+1, j] <= point: continue
        #Check if left is lower
        if j != 0 and data[i, j-1] <= point: continue
        # Check if right is lower
        if j != (data.shape[1] - 1) and data[i, j+1] <= point: continue
        low_points.append(point)
        low_point_positions.append((i, j))

In [ ]:
sum(np.asarray(low_points) + 1)

In [ ]:
basin_sizes = []
explored_areas = np.zeros_like(data)
explored_areas[data == 9] = 1
for low_point in low_point_positions:
    seed_points = set([low_point])
    size = 0
    while len(seed_points) > 0:
        #print(seed_points)
        i, j = seed_points.pop()
        explored_areas[i, j] = 1
        size += 1
        if i != 0 and explored_areas[i-1, j] != 1: seed_points.add((i-1, j))
        if i != data.shape[0] - 1 and explored_areas[i+1, j] != 1: seed_points.add((i+1, j))
        if j != 0 and explored_areas[i, j-1] != 1: seed_points.add((i, j-1))
        if j != data.shape[1] - 1 and explored_areas[i, j+1] != 1: seed_points.add((i, j+1))

    basin_sizes.append(size)
    
basin_sizes.sort()

In [ ]:
basin_sizes[-5:]

In [ ]:
basin_sizes[-1] * basin_sizes[-2] * basin_sizes[-3]

# Day 10

In [ ]:
puzzle = Puzzle(year=2021, day=10)
data = lines(puzzle.input_data)
data[:5]

In [ ]:
data = ["[({(<(())[]>[[{[]{<()<>>",
"[(()[<>])]({[<{<<[]>>(",
"{([(<{}[<>[]}>{[]{[(<()>",
"(((({<>}<{<{<>}{[]{[]{}",
"[[<[([]))<([[{}[[()]]]",
"[{[{({}]{}}([{[{{{}}([]",
"{<[[]]>}<{[{[{[]{()[[[]",
"[<(<(<(<{}))><([]([]()",
"<{([([[(<>()){}]>(<<{{",
"<{([{{}}[<[[[<>{}]]]>[]]"]

In [ ]:
opening_characters = "{([<"
closing_characters = "})]>"

opening_closing_character_mapping = {"{":"}", "(":")", "[":"]", "<":">"}

score = 0

for line in data:
    while len(line) > 1:
        line = line.replace("()", "")
        line = line.replace("[]", "")
        line = line.replace("{}", "")
        line = line.replace("<>", "")

        if "()" not in line and "{}" not in line and"[]" not in line and "<>" not in line: break
    """
    if len(line) == 0: print("Line is complete and non corrupt")
    elif len(line) > 1 and (all(map(lambda x: x in opening_characters, line)) or all(map(lambda x: x in closing_characters, line))): print("Line is incomplete")
    else: 
        print("Line is corrupt")
    """
    if not all(map(lambda x: x in opening_characters, line)):
        queue = []
        for character in line:
            if character in opening_characters: queue.append(character)
            else:
                if character == opening_closing_character_mapping[queue[-1]]: queue.pop()
                else: 
                    #print("First illegal character: ", character)
                    if character == ")": score += 3
                    if character == "]": score += 57
                    if character == "}": score += 1197
                    if character == ">": score += 25137
                    break
                
print(score)

In [ ]:
opening_characters = "{([<"
closing_characters = "})]>"

opening_closing_character_mapping = {"{":"}", "(":")", "[":"]", "<":">"}

scores = []

for line in data:
    while len(line) > 1:
        line = line.replace("()", "")
        line = line.replace("[]", "")
        line = line.replace("{}", "")
        line = line.replace("<>", "")

        if "()" not in line and "{}" not in line and"[]" not in line and "<>" not in line: break

    if len(line) == 0: continue
    if all(map(lambda x: x in opening_characters, line)):
        score = 0
        for c in line[::-1]:
            score *= 5
            if c == '(': score += 1
            if c == '[': score += 2
            if c == '{': score += 3
            if c == '<': score += 4
            
        scores.append(score)

In [ ]:
np.median(scores)

# Day 11

In [ ]:
data = ['5483143223',
'2745854711',
'5264556173',
'6141336146',
'6357385478',
'4167524645',
'2176841721',
'6882881134',
'4846848554',
'5283751526']

In [ ]:
puzzle = Puzzle(year=2021, day=11)
data = lines(puzzle.input_data)
data[:5]

In [ ]:
energy_levels = np.asarray([[int(d) for d in line] for line in data])
energy_levels

In [ ]:
n_steps = 100
current_energy_level = energy_levels.copy()
flash_count = 0
for step in range(n_steps):
    current_energy_level += 1
    i, j = np.where(current_energy_level > 9)
    seed_points = set(zip(list(i), list(j)))
    flashed = np.zeros_like(current_energy_level)
    while len(seed_points) > 0:
        i, j = seed_points.pop()
        flashed[i, j] = 1

        # Increase each neighbour and check if it becomes > 9
        for x in range(max(i-1, 0), min(i+1, current_energy_level.shape[0] - 1) + 1):
            for y in range(max(j-1, 0), min(j+1, current_energy_level.shape[1] - 1) + 1):
                current_energy_level[x, y] += 1
                if current_energy_level[x, y] > 9 and flashed[x, y] == 0: seed_points.add((x, y))
    current_energy_level[flashed == 1] = 0
    flash_count += flashed.sum()
    
flash_count

In [ ]:
current_energy_level = energy_levels.copy()
step_count = 0
flashed = np.zeros_like(current_energy_level)
while not flashed.all():
    current_energy_level += 1
    i, j = np.where(current_energy_level > 9)
    seed_points = set(zip(list(i), list(j)))
    flashed = np.zeros_like(current_energy_level)
    while len(seed_points) > 0:
        i, j = seed_points.pop()
        flashed[i, j] = 1

        # Increase each neighbour and check if it becomes > 9
        for x in range(max(i-1, 0), min(i+1, current_energy_level.shape[0] - 1) + 1):
            for y in range(max(j-1, 0), min(j+1, current_energy_level.shape[1] - 1) + 1):
                current_energy_level[x, y] += 1
                if current_energy_level[x, y] > 9 and flashed[x, y] == 0: seed_points.add((x, y))
    current_energy_level[flashed == 1] = 0
    step_count += 1
    
step_count

# Day 12

In [ ]:
puzzle = Puzzle(year=2021, day=12)
data = lines(puzzle.input_data)
data[:5]

In [ ]:
data = ["start-A",
"start-b",
"A-c",
"A-b",
"b-d",
"A-end",
"b-end"]

In [ ]:
data = ["dc-end",
"HN-start",
"start-kj",
"dc-start",
"dc-HN",
"LN-dc",
"HN-end",
"kj-sa",
"kj-HN",
"kj-dc"]

In [ ]:
data = ["fs-end",
"he-DX",
"fs-he",
"start-DX",
"pj-DX",
"end-zg",
"zg-sl",
"zg-pj",
"pj-he",
"RW-he",
"fs-DX",
"pj-RW",
"zg-RW",
"start-pj",
"he-WI",
"zg-he",
"pj-fs",
"start-RW"]

In [ ]:
all_possible_paths = []
start_points = [point for point in data if "start" in point]
start_points

In [ ]:
def find_visitable_points(start_point, data):
    available_paths = [point for point in data if start_point + "-" in point or "-" + start_point in point]
    visitable_points = set([point.replace(start_point, "").replace("-", "") for point in available_paths])
    visitable_points = [point for point in visitable_points if point != 'start'] # Don't want to go back to the start
    return visitable_points

In [ ]:
from collections import Counter
open_paths = [["start", p] for p in find_visitable_points("start", data)]
closed_paths = []

while len(open_paths) > 0:
    working_path = open_paths.pop()
   
    # check if there is any small cave being visited twice and ignore it if that's the case
    cave_visit_count = Counter(working_path)
    if any([k for k,v in cave_visit_count.items() if v > 1 and k.islower()]):
        continue
    
    # If we've already reached the end  append the path to the closed_paths list
    if working_path[-1] == 'end':
         closed_paths.append(working_path)
    else:
        visitable_points = find_visitable_points(working_path[-1], data)
        open_paths += [working_path.copy() + [p] for p in visitable_points]

In [ ]:
closed_paths 

In [ ]:
len(closed_paths)

In [ ]:
from collections import Counter
open_paths = [["start", p] for p in find_visitable_points("start", data)]
closed_paths = []

while len(open_paths) > 0:
    working_path = open_paths.pop()
   
    # check if there is any small cave being visited twice and ignore it if that's the case
    small_cave_visit_count = {k:v for k,v in Counter(working_path).items() if k.islower()}
    if max(small_cave_visit_count.values()) > 2 or sum(small_cave_visit_count.values()) > len(small_cave_visit_count.values()) + 1:
        continue
    
    # If we've already reached the end  append the path to the closed_paths list
    if working_path[-1] == 'end':
         closed_paths.append(working_path)
    else:
        visitable_points = find_visitable_points(working_path[-1], data)
        open_paths += [working_path.copy() + [p] for p in visitable_points]

In [ ]:
len(closed_paths)

In [ ]:
def find_paths(current_node = 'start', path = []):
    
    path.append(current_node)
    
    if current_node == 'end': 
        all_paths.append(path)
        return
    
    small_cave_visit_count = {k:v for k,v in Counter(path).items() if k.islower()}
    if max(small_cave_visit_count.values()) > 2 or sum(small_cave_visit_count.values()) > len(small_cave_visit_count.values()) + 1:
        return
    
    visitable_points = find_visitable_points(current_node, data)
    for point in visitable_points:
        find_paths(point, path.copy())

In [ ]:
all_paths = []
find_paths()
len(all_paths)

# Day 13

In [ ]:
puzzle = Puzzle(year=2021, day=13)
data = lines(puzzle.input_data)
data[:5]

In [ ]:
data[-2:]

In [ ]:
data = ['6,10',
'0,14',
'9,10',
'0,3',
'10,4',
'4,11',
'6,0',
'6,12',
'4,1',
'0,13',
'10,12',
'3,4',
'3,0',
'8,4',
'1,10',
'2,14',
'8,10',
'9,0',
'',
'fold along y=7',
'fold along x=5']

In [ ]:
point_coordinates = [line.split(',') for line in data if 'fold' not in line and line != '']
point_coordinates = np.asarray(point_coordinates).astype(int)
folds = [line.replace('fold along ', '').split('=') for line in data if 'fold' in line]
point_coordinates[:5]

In [ ]:
folds[:5]

In [ ]:
(point_coordinates[:, 1].max() + 1) % 2

In [ ]:
n_rows = point_coordinates[:, 1].max() + 1
if n_rows % 2 == 0: n_rows += 1
n_cols = point_coordinates[:, 0].max() + 1
if n_cols % 2 == 0: n_cols += 1
paper = np.zeros((n_rows, n_cols))
for x, y in point_coordinates:
    paper[y, x] = 1

for fold_direction, fold_line in folds [:1]:
    print(fold_direction, fold_line)
    print(paper.shape)
    fold_line = int(fold_line)
    if fold_direction == 'y':
        section1 = paper[:fold_line, :]
        section2 = paper[fold_line + 1:, :]
        paper = section1 + np.flip(section2, 0)
    else:
        section1 = paper[:, :fold_line]
        section2 = paper[:, fold_line+1:]
        paper = section1 + np.flip(section2, 1)

In [ ]:
(paper >= 1).sum()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
import seaborn as sns 
sns.heatmap(paper >= 1)

# Day 14

In [ ]:
import re
from collections import Counter
from itertools import product
puzzle = Puzzle(year=2021, day=14)
data = lines(puzzle.input_data)
data[:5]

In [ ]:
data = ['NNCB',
'',
'CH -> B',
'HH -> N',
'CB -> H',
'NH -> C',
'HB -> C',
'HC -> B',
'HN -> C',
'NN -> C',
'BH -> H',
'NC -> B',
'NB -> B',
'BN -> B',
'BB -> N',
'BC -> B',
'CC -> N',
'CN -> C']

In [ ]:
polymer_input_score_board, output_score_boardlate = data[0]
pair_insertion_rules = {rule[:2]:rule[-1] for rule in data[2:]}
polymer_input_score_board, output_score_boardlate

In [ ]:
polymer_input_score_board, output_score_boardlate = data[0]
n_steps = 40

for step in range(n_steps):
    
    print(step)

    letter_has_been_added_now = np.zeros(len(polymer_input_score_board, output_score_boardlate)).astype(bool).tolist()
    for input_letters, letter_to_add in pair_insertion_rules:

        matches = re.finditer(f'(?=({input_letters}))', polymer_input_score_board, output_score_boardlate)
        offset = 0
        for match in matches:

            # If this match is not considering a letter that has been added in this round
            if not (letter_has_been_added_now[match.start(1) + offset] or letter_has_been_added_now[match.start(1) + 1 + offset]):
                new_letter_position = match.start(1) + 1 + offset

                # Add letter to polymer input_score_board, output_score_boardlate
                polymer_input_score_board, output_score_boardlate = polymer_input_score_board, output_score_boardlate[:new_letter_position] + letter_to_add + polymer_input_score_board, output_score_boardlate[new_letter_position:]

                # Register that this letter has been added in tis round
                letter_has_been_added_now.insert(new_letter_position, True)
                offset += 1
                
    #print(polymer_input_score_board, output_score_boardlate)

In [ ]:
%%time
polymer_input_score_board, output_score_boardlate = data[0]
pair_insertion_rules = {rule[:2]:rule[-1] for rule in data[2:]}

all_possible_letters = set(pair_insertion_rules.values())
all_possible_pairs = [''.join(pair) for pair in product(all_possible_letters, repeat = 2)]

pair_counts = pd.Series(np.zeros(len(all_possible_pairs)), index = all_possible_pairs)
letter_counts = pd.Series(np.zeros(len(all_possible_letters)), index = all_possible_letters)

for letter in polymer_input_score_board, output_score_boardlate: letter_counts[letter] += 1
for letter_pair in [polymer_input_score_board, output_score_boardlate[i: i+2] for i in range(len(polymer_input_score_board, output_score_boardlate) - 1)]:
    pair_counts[letter_pair] += 1

n_steps = 40
for step in range(n_steps):
    input_score_board, pair_counts.copy()
    for input_letters, n in pair_counts[pair_counts > 0].iteritems():
        letter_to_add = pair_insertion_rules[input_letters]
        input_score_board, output_score_board[input_letters] -= n
        input_score_board, output_score_board[input_letters[0] + letter_to_add] += n
        input_score_board, output_score_board[letter_to_add + input_letters[1]] += n
        letter_counts[letter_to_add] += n
    pair_counts = input_score_board, output_score_board
    #print(step, pair_counts)
    #print(letter_counts.sum())

In [ ]:
letter_counts.max() - letter_counts.min()

# Day 15

In [ ]:
puzzle = Puzzle(year=2021, day=15)
data = lines(puzzle.input_data)
data[:5]

In [ ]:
data = ['1163751742',
'1381373672',
'2136511328',
'3694931569',
'7463417111',
'1319128137',
'1359912421',
'3125421639',
'1293138521',
'2311944581']

In [ ]:
risk_level = np.asarray([[int(digit) for digit in line] for line in data])
risk_level.shape

In [ ]:
def calculate_shortest_path_distance(risk_level):
    start_position = (0, 0)
    next_positions_to_visit = {(0, 1): risk_level[0, 1], (1, 0): risk_level[1, 0]}

    node_has_been_visited = np.zeros_like(risk_level)
    node_has_been_visited[0, 0] = 1
    cumulative_distances = np.zeros_like(risk_level)
    cumulative_distances[(0, 1)] = risk_level[0, 1]
    cumulative_distances[(1, 0)] = risk_level[1, 0]

    while len(next_positions_to_visit) > 0 and cumulative_distances[-1, -1] == 0:

        node_w_smallest_dist = min(next_positions_to_visit, key=next_positions_to_visit.get)
        current_distance = next_positions_to_visit[node_w_smallest_dist]

        # Mark as visited
        node_has_been_visited[node_w_smallest_dist] = 1

        if node_w_smallest_dist[0] != risk_level.shape[0] - 1:
            bottom_adjacent_coords = (node_w_smallest_dist[0] + 1, node_w_smallest_dist[1])
            if not(node_has_been_visited[bottom_adjacent_coords] or cumulative_distances[bottom_adjacent_coords] != 0): 
                cumulative_distances[bottom_adjacent_coords] = current_distance + risk_level[bottom_adjacent_coords]
                next_positions_to_visit[bottom_adjacent_coords] = current_distance + risk_level[bottom_adjacent_coords]
        if node_w_smallest_dist[1] != risk_level.shape[1] - 1:
            right_adjacent_coords = (node_w_smallest_dist[0], node_w_smallest_dist[1] + 1)
            if not(node_has_been_visited[right_adjacent_coords] or cumulative_distances[right_adjacent_coords] != 0): 
                cumulative_distances[right_adjacent_coords] = current_distance + risk_level[right_adjacent_coords]
                next_positions_to_visit[right_adjacent_coords] = current_distance + risk_level[right_adjacent_coords]
        if node_w_smallest_dist[0] != 0:
            top_adjacent_coords = (node_w_smallest_dist[0] - 1, node_w_smallest_dist[1])
            if not(node_has_been_visited[top_adjacent_coords] or cumulative_distances[top_adjacent_coords] != 0): 
                cumulative_distances[top_adjacent_coords] = current_distance + risk_level[top_adjacent_coords]
                next_positions_to_visit[top_adjacent_coords] = current_distance + risk_level[top_adjacent_coords]
        if node_w_smallest_dist[1] != 0:
            left_adjacent_coords = (node_w_smallest_dist[0], node_w_smallest_dist[1] - 1)
            if not(node_has_been_visited[left_adjacent_coords] or cumulative_distances[left_adjacent_coords] != 0): 
                cumulative_distances[left_adjacent_coords] = current_distance + risk_level[left_adjacent_coords]
                next_positions_to_visit[left_adjacent_coords] = current_distance + risk_level[left_adjacent_coords]


        # Remove from "to-visit" dict
        next_positions_to_visit.pop(node_w_smallest_dist)

        #print(node_w_smallest_dist)
        #print(cumulative_distances)
        #print(next_positions_to_visit)
        #print()
        
    return cumulative_distances[-1, -1]

In [ ]:
calculate_shortest_path_distance(risk_level)

In [ ]:
n_rows, n_cols = risk_level.shape
large_risk_level = np.zeros((n_rows*5, n_cols*5))

current = risk_level.copy()

for i in range(5):
    large_risk_level[i*n_rows:(i+1)*n_rows, :n_cols] = current
    current += 1
    current[current > 9] = 1
    
current = large_risk_level[:, :n_cols].copy()
for j in range(5):
    large_risk_level[:, j*n_cols:(j+1)*n_cols] = current
    current += 1
    current[current > 9] = 1
    
large_risk_level = large_risk_level.astype(int)

In [ ]:
calculate_shortest_path_distance(large_risk_level)

# Day 16

In [ ]:
puzzle = Puzzle(year=2021, day=16)
data = puzzle.input_data
data

In [ ]:
class Tokenizer:
    
    def __init__(self, binary_packet):
        self.read_pointer = 0
        self.binary_packet = binary_packet
    
    def get_version_and_type(self):
        packet_version = int(self.binary_packet[self.read_pointer:self.read_pointer+3], 2)
        packet_type_id = int(self.binary_packet[self.read_pointer+3:self.read_pointer+6], 2)
        self.read_pointer += 6
        return packet_version, packet_type_id
    
    def get_operator_packet_type(self):
        operator_packet_type = self.binary_packet[self.read_pointer]
        self.read_pointer += 1
        return int(operator_packet_type)
    
    def get_subpackets_length(self):
        length = int(self.binary_packet[self.read_pointer:self.read_pointer + 15], 2)
        self.read_pointer += 15
        return length
        
    def get_n_subpackets(self):
        n = int(self.binary_packet[self.read_pointer:self.read_pointer + 11], 2)
        self.read_pointer += 11
        return n        
    
    def get_subpackets_w_length(self, length):
        return self.binary_packet[self.read_pointer: self.read_pointer+length]

    def get_remaining_packet(self):
        return self.binary_packet[self.read_pointer:]
    
    def decode_packet(self):
        
        if len(self.binary_packet.replace('0', '')) == 0: return
    
        packet_version, packet_type_id = self.get_version_and_type()
        versions.append(packet_version)
        
        #print(versions)
        #print(packet_type_id)

        if packet_type_id == 4:
            literal = self.decode_literal_packet()
            #print("Literal:", literal)
        else:
            operator_type = self.get_operator_packet_type()
            #print("Operator type:", operator_type)
            if operator_type == 0:
                self.decode_fixed_length_packet()
            else:
                self.decode_packet_w_n_subpackets()
    
    def decode_literal_packet(self):
        
        group_prefix = self.binary_packet[self.read_pointer]
    
        digits = []

        while group_prefix != '0':
            digits.append(self.binary_packet[self.read_pointer + 1: self.read_pointer + 5])
            self.read_pointer += 5
            group_prefix = self.binary_packet[self.read_pointer]

        # Append final digit 
        digits.append(self.binary_packet[self.read_pointer + 1: self.read_pointer + 5])
        self.read_pointer += 5

        decimal_value = int(''.join(digits), 2)
        
        return decimal_value
    
    def decode_fixed_length_packet(self):
        subpackets_length = self.get_subpackets_length()
        #print("Subpackets length:", subpackets_length)
        subtokenizer = Tokenizer(self.binary_packet[self.read_pointer: self.read_pointer + subpackets_length])
        subtokenizer.decode_packet()
        self.read_pointer += subtokenizer.read_pointer
        subpackets_length -= subtokenizer.read_pointer
        
        while len(subtokenizer.get_remaining_packet().replace('0', '')) > 0:
            subtokenizer = Tokenizer(self.binary_packet[self.read_pointer: self.read_pointer + subpackets_length])
            subtokenizer.decode_packet()
            self.read_pointer += subtokenizer.read_pointer
            subpackets_length -= subtokenizer.read_pointer
        
    def decode_packet_w_n_subpackets(self):
        n_subpackets = self.get_n_subpackets()
        #print("Number of subpackets:", n_subpackets)
        
        for i in range(n_subpackets):
            subtokenizer = Tokenizer(self.get_remaining_packet())
            subtokenizer.decode_packet()
            self.read_pointer += subtokenizer.read_pointer

In [ ]:
hex_packet = '38006F45291200'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

versions = [] 
tokenizer = Tokenizer(binary_packet)

tokenizer.decode_packet()

print("Final list of versions:", versions)

In [ ]:
hex_packet = 'EE00D40C823060'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

versions = [] 

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print("Final list of versions:", versions)

In [ ]:
hex_packet = '8A004A801A8002F478'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

versions = [] 

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print("Final list of versions:", versions)

In [ ]:
hex_packet = '620080001611562C8802118E34'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

versions = [] 

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print("Final list of versions:", versions)
print("Version sum:", sum(versions))

In [ ]:
hex_packet = 'C0015000016115A2E0802F182340'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

versions = [] 

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print("Final list of versions:", versions)
print("Version sum:", sum(versions))

In [ ]:
hex_packet = 'A0016C880162017C3686B18A3D4780'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

versions = [] 

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print("Final list of versions:", versions)
print("Version sum:", sum(versions))

In [ ]:
hex_packet = data
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

versions = [] 

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print("Final list of versions:", versions)
print("Version sum:", sum(versions))

In [ ]:
# Type 0: sum
# 1: product
# 2: minimum
# 3: maximum
# 4: literal
# 5: greater than
# 6: less than
# 7: equal to

In [ ]:
class Tokenizer:
    
    def __init__(self, binary_packet):
        self.read_pointer = 0
        self.binary_packet = binary_packet
    
    def get_version_and_type(self):
        packet_version = int(self.binary_packet[self.read_pointer:self.read_pointer+3], 2)
        packet_type_id = int(self.binary_packet[self.read_pointer+3:self.read_pointer+6], 2)
        self.read_pointer += 6
        return packet_version, packet_type_id
    
    def get_operator_packet_type(self):
        operator_packet_type = self.binary_packet[self.read_pointer]
        self.read_pointer += 1
        return int(operator_packet_type)
    
    def get_subpackets_length(self):
        length = int(self.binary_packet[self.read_pointer:self.read_pointer + 15], 2)
        self.read_pointer += 15
        return length
        
    def get_n_subpackets(self):
        n = int(self.binary_packet[self.read_pointer:self.read_pointer + 11], 2)
        self.read_pointer += 11
        return n        
    
    def get_subpackets_w_length(self, length):
        return self.binary_packet[self.read_pointer: self.read_pointer+length]

    def get_remaining_packet(self):
        return self.binary_packet[self.read_pointer:]
    
    def decode_packet(self):
        
        if len(self.binary_packet.replace('0', '')) == 0: return
    
        packet_version, packet_type_id = self.get_version_and_type()
        #versions.append(packet_version)
        
        #print(versions)
        #print(packet_type_id)

        if packet_type_id == 4:
            self.decode_literal_packet()
            #print("Literal:", literal)
        else:
            
            # Find which operation this is 
            if packet_type_id == 0: operation = '+'
            elif packet_type_id == 1: operation = '*'
            elif packet_type_id == 2: operation = 'min'
            elif packet_type_id == 3: operation = 'max'
            elif packet_type_id == 5: operation = '>'
            elif packet_type_id == 6: operation = '<'
            elif packet_type_id == 7: operation = '=='
            
            operator_type = self.get_operator_packet_type()
            #print("Operator type:", operator_type)
            if operator_type == 0:
                self.decode_fixed_length_packet(operation)
            else:
                self.decode_packet_w_n_subpackets(operation)
    
    def decode_literal_packet(self):
        
        group_prefix = self.binary_packet[self.read_pointer]
    
        digits = []

        while group_prefix != '0':
            digits.append(self.binary_packet[self.read_pointer + 1: self.read_pointer + 5])
            self.read_pointer += 5
            group_prefix = self.binary_packet[self.read_pointer]

        # Append final digit 
        digits.append(self.binary_packet[self.read_pointer + 1: self.read_pointer + 5])
        self.read_pointer += 5

        decimal_value = int(''.join(digits), 2)
        
        string_builder.append(str(decimal_value))
    
    def decode_fixed_length_packet(self, operation):
        
        if operation in ['min', 'max']: 
            string_builder.append(operation)
        string_builder.append("(")
        
        #print(string_builder)
        
        subpackets_length = self.get_subpackets_length()
        #print("Subpackets length:", subpackets_length)
        subtokenizer = Tokenizer(self.binary_packet[self.read_pointer: self.read_pointer + subpackets_length])
        subtokenizer.decode_packet()
        self.read_pointer += subtokenizer.read_pointer
        subpackets_length -= subtokenizer.read_pointer
        
        if operation in ['min', 'max']: string_builder.append(',')
        else: string_builder.append(operation)
        
        while len(subtokenizer.get_remaining_packet().replace('0', '')) > 0:
            subtokenizer = Tokenizer(self.binary_packet[self.read_pointer: self.read_pointer + subpackets_length])
            subtokenizer.decode_packet()
            self.read_pointer += subtokenizer.read_pointer
            subpackets_length -= subtokenizer.read_pointer
            
            if operation in ['min', 'max']: string_builder.append(',')
            else: string_builder.append(operation)
            
        string_builder.pop(-1)
                             
        string_builder.append(")")
        
    def decode_packet_w_n_subpackets(self, operation):
        
        if operation in ['min', 'max']: 
            string_builder.append(operation)
        string_builder.append("(")
        
        n_subpackets = self.get_n_subpackets()
        #print("Number of subpackets:", n_subpackets)
        
        for i in range(n_subpackets):
            subtokenizer = Tokenizer(self.get_remaining_packet())
            subtokenizer.decode_packet()
            self.read_pointer += subtokenizer.read_pointer
            
            if operation in ['min', 'max']: string_builder.append(',')
            else: string_builder.append(operation)
            
        string_builder.pop(-1)
        
        string_builder.append(")")

In [ ]:
hex_packet = '880086C3E88112'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))
print(eval(''.join(string_builder)))

In [ ]:
hex_packet = 'CE00C43D881120'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))
print(eval(''.join(string_builder)))

In [ ]:
hex_packet = 'C200B40A82'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))
print(eval(''.join(string_builder)))

In [ ]:
hex_packet = '04005AC33890'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))
print(eval(''.join(string_builder)))

In [ ]:
hex_packet = 'D8005AC2A8F0'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))
print(eval(''.join(string_builder)))

In [ ]:
hex_packet = 'F600BC2D8F'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))
print(eval(''.join(string_builder)))

In [ ]:
hex_packet = '9C005AC2F8F0'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))
print(eval(''.join(string_builder)))

In [ ]:
hex_packet = '9C0141080250320F1802104A08'
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))
print(eval(''.join(string_builder)))

In [ ]:
hex_packet = data
h_size = len(hex_packet) * 4
binary_packet = bin(int(hex_packet, 16))[2:].zfill(h_size)

string_builder = []

tokenizer = Tokenizer(binary_packet)
tokenizer.decode_packet()

print(string_builder)
print(''.join(string_builder))

In [ ]:
full_calculation = ''.join(string_builder)
import re

full_calculation = re.sub('min\((\d+)\)', r'\1', full_calculation)
full_calculation = re.sub('max\((\d+)\)', r'\1', full_calculation)

while re.search('min\((\d+)\)', full_calculation) or re.search('max\((\d+)\)', full_calculation) or re.search('\(\((\d+)\)\)', full_calculation):
    full_calculation = re.sub('min\((\d+)\)', r'\1', full_calculation)
    full_calculation = re.sub('max\((\d+)\)', r'\1', full_calculation)
    
    full_calculation = re.sub('\((\(\d+\))\)', r'\1', full_calculation)

In [ ]:
eval(full_calculation)

# Day 17

In [ ]:
puzzle = Puzzle(year=2021, day=17)
data = puzzle.input_data
data

In [ ]:
xmin = int(data[15:18])
xmax = int(data[20:23])
ymin = int(data[27:31])
ymax = int(data[33:37])

In [ ]:
xmin, xmax, ymin, ymax

In [ ]:
data = 'target area: x=20..30, y=-10..-5'
xmin = 20
xmax = 30
ymin = -10
ymax = -5

In [ ]:
max_height = 0
best_v0x = 0
best_v0y = 0
for v0x in range(xmax + 1):
    for v0y in range(1000):
        vnx = v0x
        vny = v0y
        pnx = 0
        pny = 0
        for n in range(1000):
            pnx = pnx + vnx
            pny = pny + vny
            
            if vnx < 0: vnx += 1
            elif vnx > 0: vnx -= 1
            vny -= 1
            
            if pnx >= xmin and pnx <= xmax and pny >= ymin and pny <= ymax:
                print(f"Hit the box for initial velocities: {v0x}, {v0y} at step {n}")
                print(f"Positions: {pnx}, {pny}")
                highest_point = v0y*v0y - v0y * (v0y - 1) / 2
                print("Height reached:", highest_point)
                if max_height < highest_point:
                    max_height = highest_point
                    best_v0x = v0x
                    best_v0y = v0y
                    
                break
                    
            if pnx > xmax or pny < ymin: break

In [ ]:
max_height

In [ ]:
max_height = 0
best_v0x = 0
best_v0y = 0

count = 0

for v0x in range(xmax + 1):
    for v0y in range(-1000, 1000):
        vnx = v0x
        vny = v0y
        pnx = 0
        pny = 0
        for n in range(1000):
            pnx = pnx + vnx
            pny = pny + vny
            
            if vnx < 0: vnx += 1
            elif vnx > 0: vnx -= 1
            vny -= 1
            
            if pnx >= xmin and pnx <= xmax and pny >= ymin and pny <= ymax:
                #print(f"Hit the box for initial velocities: {v0x}, {v0y} at step {n}")
                count += 1
                break
                    
            if pnx > xmax or pny < ymin: break

In [ ]:
count

# Day 18

In [ ]:
puzzle = Puzzle(year=2021, day=18)
data = lines(puzzle.input_data)
data

In [ ]:
class Node:
    def __init__(self, left_node = None, right_node = None, parent_node = None):

        self.parent_node = parent_node
        self.left_node = left_node
        self.right_node = right_node
        
        self.is_right_node = False
        self.is_left_node = False
        
        if type(left_node) != int:
            self.left_node.set_as_left_node()
            self.left_node.parent_node = self
        if type(right_node) != int:
            self.right_node.set_as_right_node()
            self.right_node.parent_node = self
            
    def set_as_right_node(self):
        self.is_right_node = True
        self.is_left_node = False
    
    def set_as_left_node(self):
        self.is_right_node = False
        self.is_left_node = True
        
class Tree:
    def __init__(self, root_node):
        self.root = root_node 
    
    def add_node_to_right(self, new_node):
        self.root = Node(self.root, new_node)
    
    def add_node_to_left(self, new_node):
        self.root = Node(new_node, self.root)
        
    def to_list_from_root(self):
        return to_list(self.root)
        
    def _to_list(self, base_node):

        if type(base_node) == int:
            return base_node
        else:
            return [to_list(base_node.left_node), to_list(base_node.right_node)]
    
    def get_depths_from_root(self):
        return get_depths(self.root, 0)
        
    def _get_depths(self, base_node, current_depth):
        
        if type(base_node.left_node) == int:
            return current_depth
        else:
            return [get_depths(base_node.left_node, current_depth + 1), get_depths(base_node.right_node, current_depth + 1)]

    def find_node_to_right(self, node):
        
        if node.parent_node == None: return None
        
        previous_node = node
        current_node = node.parent_node
        if previous_node.is_right_node:
            return self.find_node_to_right(current_node)
        else:
            if type(current_node.right_node) == int: return current_node
            current_node = current_node.right_node
            while type(current_node.left_node) != int:
                current_node = current_node.left_node
            return current_node#.left_node
            
    def find_node_to_left(self, node):
        
        if node.parent_node == None: return None
        
        previous_node = node
        current_node = node.parent_node
        if previous_node.is_left_node:
            return self.find_node_to_left(current_node)
        else:
            if type(current_node.left_node) == int: return current_node
            current_node = current_node.left_node
            while type(current_node.right_node) != int:
                current_node = current_node.right_node
            return current_node#.right_node
        
    def explode(self):
        return explode(self.root, 0)
    
    def _explode(self, base_node, current_depth):
        
        if type(base_node) == int: return False
        
        if type(base_node.left_node) == int and type(base_node.right_node) == int and current_depth >= 4:
            
            #print(base_node.left_node, base_node.right_node)
            
            closest_left_node = self.find_node_to_left(base_node)
            if closest_left_node is not None:
                if type(closest_left_node.right_node) == int: closest_left_node.right_node += base_node.left_node
                else: closest_left_node.left_node += base_node.left_node
            closest_right_node = self.find_node_to_right(base_node)
            if closest_right_node is not None:
                #print("Closest right node:", closest_right_node.left_node, closest_right_node.right_node)
                if type(closest_right_node.left_node) == int: closest_right_node.left_node += base_node.right_node
                else: closest_right_node.right_node += base_node.right_node
            
            # Replace this node with the value 0
            if base_node.is_left_node:
                base_node.parent_node.left_node = 0
            else: 
                base_node.parent_node.right_node = 0
            return True
            
        elif explode(base_node.left_node, current_depth + 1):
            return True
        elif explode(base_node.right_node, current_depth + 1):
            return True
        else: return False
    
    def split(self):
        return split(self.root)
    
    def _split(self, base_node):
        if type(base_node.left_node) == int:
            #print("Left node:", base_node.left_node)
            if base_node.left_node >= 10:
                #print("Splitting")
                # Split left node
                left_value = int(np.floor(base_node.left_node / 2))
                right_value = int(np.ceil(base_node.left_node / 2))
                base_node.left_node = Node(left_value, right_value)
                base_node.left_node.parent_node = base_node
                base_node.left_node.set_as_left_node()
                return True
            elif type(base_node.right_node) == int:
                #print("Right node:", base_node.right_node)
                if base_node.right_node >= 10:
                    #print("Splitting")
                    original_value = base_node.right_node
                    # Split right node
                    left_value = int(np.floor(original_value / 2))
                    right_value = int(np.ceil(original_value / 2))
                    base_node.right_node = Node(left_value, right_value)
                    base_node.right_node.parent_node = base_node
                    base_node.right_node.set_as_right_node()
                    return True
        
        elif split(base_node.left_node): return True
        if type(base_node.right_node) == int:
            #print("Right node:", base_node.right_node)
            if base_node.right_node >= 10:
                #print("Splitting")
                # Split right node
                original_value = base_node.right_node 
                left_value = int(np.floor(original_value / 2))
                right_value = int(np.ceil(original_value / 2))
                base_node.right_node = Node(left_value, right_value)
                base_node.right_node.parent_node = base_node
                base_node.right_node.set_as_right_node()
                return True
        elif split(base_node.right_node): return True
        return False
    
    def compute_magnitude(self):
        return compute_magnitude(self.root)
    
    def _compute_magnitude(self, base_node):
        
        if type(base_node.left_node) == int: left_magnitude = 3*base_node.left_node
        else: left_magnitude = 3*compute_magnitude(base_node.left_node)
        if type(base_node.right_node) == int: right_magnitude = 2*base_node.right_node
        else: right_magnitude = 2*compute_magnitude(base_node.right_node)
        
        return left_magnitude + right_magnitude
        
    
def create_node_structure_from_string(s):
    t = eval(s.replace('[', 'Node(').replace(']', ')'))
    return t

def create_tree_from_string(s):
    return Tree(create_node_structure_from_string(s))

In [ ]:
t = Tree(Node(1, 1))
print(t.to_list_from_root())
t.add_node_to_right(Node(2, 2))
t.add_node_to_right(Node(3, 3))
t.add_node_to_right(Node(4, 4))
t.add_node_to_right(Node(5, 5))
t.add_node_to_right(Node(6, 6))

print(t.to_list_from_root())

while t.explode():
    print(t.to_list_from_root())

In [ ]:
import re
i = '[[[[4,3],4],4],[7,[[8,4],9]]]'
t = create_tree_from_string(i)

print(t.to_list_from_root())

t.add_node_to_right(Node(1, 1))
print(t.to_list_from_root())

while True:
    if t.explode(): 
        print("Exploded")
        print(t.to_list_from_root())
        continue
    if t.split():
        print("Split")
        print(t.to_list_from_root())
        continue
    break

In [ ]:
puzzle = Puzzle(year=2021, day=18)
data = lines(puzzle.input_data)

t = create_tree_from_string(data[0])

print(t.to_list_from_root())

for i in range(1, len(data)):
    print("Step", i)
    t.add_node_to_right(create_node_structure_from_string(data[i]))
    #print(t.to_list_from_root())

    while True:
        if t.explode(): 
            #print("Exploded")
            #print(t.to_list_from_root())
            continue
        if t.split():
            #print("Split")
            #print(t.to_list_from_root())
            continue
        break
        
print(t.to_list_from_root())

In [ ]:
t.compute_magnitude()

In [ ]:
from itertools import permutations

all_number_pairs = permutations(data, r = 2)

magnitudes = []

for number1, number2 in all_number_pairs:
    
    t = create_tree_from_string(number1)
    t.add_node_to_right(create_node_structure_from_string(number2))

    while True:
        if t.explode():continue
        if t.split():continue
        break
        
    magnitudes.append(t.compute_magnitude())

In [ ]:
max(magnitudes)

# Day 19

In [ ]:
puzzle = Puzzle(year=2021, day=19)
data = lines(puzzle.input_data)
scanner_readings = np.asarray(data)
scanner_readings[:5]

In [ ]:
data = ['--- scanner 0 ---',
'404,-588,-901',
'528,-643,409',
'-838,591,734',
'390,-675,-793',
'-537,-823,-458',
'-485,-357,347',
'-345,-311,381',
'-661,-816,-575',
'-876,649,763',
'-618,-824,-621',
'553,345,-567',
'474,580,667',
'-447,-329,318',
'-584,868,-557',
'544,-627,-890',
'564,392,-477',
'455,729,728',
'-892,524,684',
'-689,845,-530',
'423,-701,434',
'7,-33,-71',
'630,319,-379',
'443,580,662',
'-789,900,-551',
'459,-707,401',
'',
'--- scanner 1 ---',
'686,422,578',
'605,423,415',
'515,917,-361',
'-336,658,858',
'95,138,22',
'-476,619,847',
'-340,-569,-846',
'567,-361,727',
'-460,603,-452',
'669,-402,600',
'729,430,532',
'-500,-761,534',
'-322,571,750',
'-466,-666,-811',
'-429,-592,574',
'-355,545,-477',
'703,-491,-529',
'-328,-685,520',
'413,935,-424',
'-391,539,-444',
'586,-435,557',
'-364,-763,-893',
'807,-499,-711',
'755,-354,-619',
'553,889,-390',
'',
'--- scanner 2 ---',
'649,640,665',
'682,-795,504',
'-784,533,-524',
'-644,584,-595',
'-588,-843,648',
'-30,6,44',
'-674,560,763',
'500,723,-460',
'609,671,-379',
'-555,-800,653',
'-675,-892,-343',
'697,-426,-610',
'578,704,681',
'493,664,-388',
'-671,-858,530',
'-667,343,800',
'571,-461,-707',
'-138,-166,112',
'-889,563,-600',
'646,-828,498',
'640,759,510',
'-630,509,768',
'-681,-892,-333',
'673,-379,-804',
'-742,-814,-386',
'577,-820,562',
'',
'--- scanner 3 ---',
'-589,542,597',
'605,-692,669',
'-500,565,-823',
'-660,373,557',
'-458,-679,-417',
'-488,449,543',
'-626,468,-788',
'338,-750,-386',
'528,-832,-391',
'562,-778,733',
'-938,-730,414',
'543,643,-506',
'-524,371,-870',
'407,773,750',
'-104,29,83',
'378,-903,-323',
'-778,-728,485',
'426,699,580',
'-438,-605,-362',
'-469,-447,-387',
'509,732,623',
'647,635,-688',
'-868,-804,481',
'614,-800,639',
'595,780,-596',
'',
'--- scanner 4 ---',
'727,592,562',
'-293,-554,779',
'441,611,-461',
'-714,465,-776',
'-743,427,-804',
'-660,-479,-426',
'832,-632,460',
'927,-485,-438',
'408,393,-506',
'466,436,-512',
'110,16,151',
'-258,-428,682',
'-393,719,612',
'-211,-452,876',
'808,-476,-593',
'-575,615,604',
'-485,667,467',
'-680,325,-822',
'-627,-443,-432',
'872,-547,-609',
'833,512,582',
'807,604,487',
'839,-516,451',
'891,-625,532',
'-652,-548,-490',
'30,-46,-14']

scanner_readings = np.asarray(data)
scanner_readings[:5]

In [ ]:
all_scanners = []
for i in range(50):
    start = np.where(scanner_readings == f'--- scanner {i} ---')[0]
    if len(start) == 0: break
    else: start = start[0]
    end = np.where(scanner_readings == f'--- scanner {i+1} ---')[0]
    if len(end) == 0: end = len(scanner_readings)
    else: end = end[0]
    scanner = scanner_readings[start+1:end]
    scanner = [s for s in scanner if s != '']
    scanner = np.asarray([s.split(',') for s in scanner]).astype(int)
    all_scanners.append(scanner)

In [ ]:
all_rotations = [np.array([[1,  0,  0], [0,  1,  0], [0,  0,  1]]),
                 np.array([[1,  0,  0], [0,  0, -1], [0,  1,  0]]),
                 np.array([[1,  0,  0], [0, -1,  0], [0,  0, -1]]),
                 np.array([[1,  0,  0], [0,  0,  1], [0, -1,  0]]),
                 np.array([[0, -1,  0], [1,  0,  0], [0,  0,  1]]),
                 np.array([[0,  0,  1], [1,  0,  0], [0,  1,  0]]),
                 np.array([[0,  1,  0], [1,  0,  0], [0,  0, -1]]),
                 np.array([[0,  0, -1], [1,  0,  0], [0, -1,  0]]),
                 np.array([[-1, 0,  0], [0, -1,  0], [0,  0,  1]]),
                 np.array([[-1, 0,  0], [0,  0, -1], [0, -1,  0]]),
                 np.array([[-1, 0,  0], [0,  1,  0], [0,  0, -1]]),
                 np.array([[-1, 0,  0], [0,  0,  1], [0,  1,  0]]),
                 np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 1]]),
                 np.array([[0, 0, 1], [-1, 0, 0], [0, -1, 0]]),
                 np.array([[0, -1, 0], [-1, 0, 0], [0, 0, -1]]),
                 np.array([[0, 0, -1], [-1, 0, 0], [0, 1, 0]]),
                 np.array([[0, 0, -1], [0, 1, 0], [1, 0, 0]]),
                 np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]),
                 np.array([[0, 0, 1], [0, -1, 0], [1, 0, 0]]),
                 np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]]),
                 np.array([[0, 0, -1], [0, -1, 0], [-1, 0, 0]]),
                 np.array([[0, -1, 0], [0, 0, 1], [-1, 0, 0]]),
                 np.array([[0, 0, 1], [0, 1, 0], [-1, 0, 0]]),
                 np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]])]

#all_rotations = all_rotations + [-r for r in all_rotations]
len(all_rotations)

In [ ]:
def propagate_transformations(scanner_idx_0):

    while len(scanner_rotations[scanner_idx_0].keys()) < len(all_scanners):
        for scanner_idx_0 in [i for i in range(len(all_scanners)) if i not in scanner_translations[0].keys()] + [scanner_idx_0]:# range(len(all_scanners)):
            neighbouring_scanner_idxs = list(scanner_rotations[scanner_idx_0].keys())
            #print(neighbouring_scanner_idxs)

            for scanner_idx_1 in  neighbouring_scanner_idxs:

                neighbouring_scanner_idxs_level2 = list(scanner_rotations[scanner_idx_1].keys())
                #print("Level 2:", neighbouring_scanner_idxs_level2)

                for scanner_idx_2 in neighbouring_scanner_idxs_level2:
                    if scanner_idx_2 in scanner_rotations[scanner_idx_0].keys(): continue
                    scanner_rotations[scanner_idx_0][scanner_idx_2] = np.matmul(scanner_rotations[scanner_idx_1][scanner_idx_2], scanner_rotations[scanner_idx_0][scanner_idx_1])
                    scanner_translations[scanner_idx_0][scanner_idx_2] = np.matmul(scanner_translations[scanner_idx_1][scanner_idx_2], scanner_rotations[scanner_idx_0][scanner_idx_1]) + scanner_translations[scanner_idx_0][scanner_idx_1]

            print(sum(sum(get_adj_mat())))
            
def get_adj_mat():
    adj_mat = np.zeros((len(all_scanners), len(all_scanners)))
    for idx0, neighbours in scanner_rotations.items():
        for idx1 in neighbours.keys():
            adj_mat[idx0, idx1] = 1
    return adj_mat

def transform_beacon_coords(from_scanner, to_scanner, coords):
    rotation = scanner_rotations[to_scanner][from_scanner]
    translation = scanner_translations[to_scanner][from_scanner]
    return np.matmul(coords, rotation) + translation

In [19]:
%%time

scanner_rotations = {i:dict() for i in range(len(all_scanners))}
scanner_translations = {i:dict() for i in range(len(all_scanners))}

all_scanners = all_scanners.copy()

for scanner_idx_0 in range(len(all_scanners) ): 
    
    scanner0 = all_scanners[scanner_idx_0]
    
    for scanner_idx_1 in range(len(all_scanners)): 
        
        if scanner_idx_0 == scanner_idx_1: continue
        
        scanner1 = all_scanners[scanner_idx_1]
        
        correct_rotation = None

        for r in all_rotations:

            rotated_scanner1 = np.matmul(scanner1, r)

            #possible_translations = [i - j for i, j in zip(scanner0, rotated_scanner1)]
            possible_translations = []
            for b0 in scanner0:
                for b1 in rotated_scanner1:
                    possible_translations.append(b0 - b1)
                    
            translation_frequencies = pd.value_counts([str(list(v)) for v in possible_translations])
            if scanner_idx_0 == 5 and scanner_idx_1 == 35: print(r, translation_frequencies)
            
            if any(translation_frequencies >= 12):
                
                translation = eval(translation_frequencies.idxmax())

                correct_rotation = r
                #print(f"Found the pairs for scanners {scanner_idx_0}, {scanner_idx_1}.")
                #print("Rotation:", r)
                #print("Translation:", translation)
                
                scanner_rotations[scanner_idx_0][scanner_idx_1] = r
                scanner_translations[scanner_idx_0][scanner_idx_1] = translation
                
                break

Wall time: 2min 42s


In [ ]:
propagate_transformations(0)

all_beacons = all_scanners[0].copy()

for scanner_idx_1 in scanner_rotations[0].keys():
    #print(scanner_idx_1)
    if scanner_idx_1 == 0: continue
    transformed_beacon_coords =  transform_beacon_coords(scanner_idx_1, 0, all_scanners[scanner_idx_1])
    all_beacons = np.concatenate((all_beacons, transformed_beacon_coords), axis = 0)

all_beacons = np.unique(all_beacons, axis = 0)

In [22]:
len(all_beacons)

483

In [23]:
manhattan_distances = []
for scanner_idx_0 in range(len(all_scanners) - 1):
    for scanner_idx_1 in range(scanner_idx_0 + 1, len(all_scanners)):
        if scanner_idx_1 not in scanner_translations[scanner_idx_0].keys():
            propagate_transformations(scanner_idx_0)
        translation = scanner_translations[scanner_idx_0][scanner_idx_1]
        manhattan_distances.append(sum(np.abs(translation)))

783.0
797.0
813.0
833.0
853.0
870.0
884.0
905.0
916.0
931.0
948.0
971.0
1006.0
1039.0
1066.0
1077.0
1088.0
1115.0
1126.0
1159.0
1170.0
1193.0
1204.0
1215.0
1242.0
1262.0
1295.0
1325.0
1336.0
1356.0
1367.0
1394.0
1420.0
1440.0
1465.0
1485.0
1496.0
1516.0
1543.0
1563.0
1600.0


In [24]:
max(manhattan_distances)

14804

# Day 20

In [74]:
puzzle = Puzzle(year=2021, day=20)
data = lines(puzzle.input_data)

In [85]:
data[0]

'#.#.##..#..#..###.#.#....#.########.#.##.#..#.###..###.##.#.##.#.#.....#..##.#.#..###.###.######..#.#..#######.#..#....####..###.####.###.#.#######.#...#...#.##.###..###..##.#.#.###........##.#.....#.##.#.####...#...#.#..###.#.#...#....#...####..#.########.#...#.####.#####..#.#.###......#.##...###..##..#.#..#....#..###.#.##.....##.#####..##.####.#.###....##.###...#.##....##.#..#.#..#..#.##...#.##..#####.####.#.##...##...##...#.##.#.#.####..##...#.....#......#.#......#..###..#..##..##.###..#####..#..##.#..#.'

In [69]:
data = ['..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..###..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#..#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#......#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#.....####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.......##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#',
'',
'#..#.',
'#....',
'##..#',
'..#..',
'..###']

In [167]:
n_steps = 50

algorithm = data[0].replace('#', '1').replace('.', '0')
input_image = [d.replace('#', '1').replace('.', '0') for d in data[2:]]
input_image = np.array([[i for i in d] for d in input_image])#.astype(int)

## THIS ONLY WORKS IF algorithm[0] == '#' and algorithm[512] = '.'
count = 0
for step in range(n_steps):
    # Add border of 2 lines around the image
    if step % 2 == 0: 
        input_score_board, np.full((input_image.shape[0] + 4, input_image.shape[1] + 4), '0')
        output_image = np.full(input_score_board, output_score_board.shape, '1')
    else:
        input_score_board, np.full((input_image.shape[0] + 4, input_image.shape[1] + 4), '1')
        output_image = np.full(input_score_board, output_score_board.shape, '0')
        
    input_score_board, output_score_board[2:2+input_image.shape[0], 2:2+input_image.shape[1]] = input_image.copy()
    input_image = input_score_board, output_score_board.copy()

    for i in range(1, input_image.shape[0]-1):
        for j in range(1, input_image.shape[1] - 1):
            
            """start_i = max(i-1, 0)
            end_i = min(i+2, input_image.shape[0])
            start_j = max(j-1, 0)
            end_j = min(j+2, input_image.shape[1])"""
            
            flattened_window = input_image[i-1:i+2, j-1:j+2].ravel()
            position = int(''.join(list(flattened_window)), 2)
            output_image[i, j] = algorithm[position]
                          
    input_image = output_image.copy()
    
    #print(input_image)

In [168]:
sum(sum(input_image.astype(int)))

20097

# Day 21

In [19]:
puzzle = Puzzle(year=2021, day=21)
data = lines(puzzle.input_data)

In [20]:
data

['Player 1 starting position: 7', 'Player 2 starting position: 4']

In [3]:
data = ['Player 1 starting position: 4',
'Player 2 starting position: 8']

In [21]:
player1_start_pos = int(data[0][-1])
player2_start_pos = int(data[1][-1])
player1_start_pos, player2_start_pos

(7, 4)

In [5]:
def make_1_move(die_pos, start_pos):
    
    rolled_die_values = [i if i <= 100 else i%100 for i in die_pos + np.array([1, 2, 3])]
    new_die_pos = rolled_die_values[-1]
    new_die_pos = new_die_pos if new_die_pos <= 100 else new_die_pos %100
    
    end_pos = (start_pos + sum(rolled_die_values)) % 10
    end_pos = 10 if end_pos == 0 else end_pos
    
    return new_die_pos, end_pos

die_pos = 0
player1_pos = player1_start_pos
player2_pos = player2_start_pos
player1_score = 0
player2_score = 0
n_rolls = 0

while True:
    
    # Player 1 plays    
    die_pos, player1_pos = make_1_move(die_pos, player1_pos)
    player1_score += player1_pos
    n_rolls += 3
    if player1_score >= 1000: break
    
    # Player 2 plays
    die_pos, player2_pos = make_1_move(die_pos, player2_pos)
    player2_score += player2_pos
    n_rolls += 3
    if player2_score >= 1000: break
    
    #print("Player 1 position:", player1_pos, "score:", player1_score)
    #print("Player 2 position:", player2_pos, "score:", player2_score)

In [489]:
loser_score = player1_score if player1_score < player2_score else player2_score
loser_score * n_rolls

739785

In [22]:
player1_win_count = 0
player2_win_count = 0

def create_new_universes(input_score_board, n_moves, n_repetitions, player1_pos, player2_pos, player1_score, player2_score, player1_moving = True):

    # Positions = [1, 10]
    n_universes_in_config = input_score_board[player1_pos - 1, player2_pos - 1, player1_score, player2_score]
    
    if n_universes_in_config == 0: 
        return #output_score_board.copy()
    #print(player1_pos, player2_pos, player1_score, player2_score,n_universes_in_config)
    
    if player1_moving: 
        new_player1_pos = player1_pos + n_moves
        new_player1_pos = new_player1_pos if new_player1_pos <= 10 else new_player1_pos % 10
        new_player1_score = player1_score + new_player1_pos
        score_board[new_player1_pos - 1, player2_pos - 1, new_player1_score, player2_score] += n_universes_in_config * n_repetitions
        #print(new_player1_pos, player2_pos, new_player1_score, player2_score,n_universes_in_config* n_repetitions)
        #print()
    else:
        new_player2_pos = player2_pos + n_moves
        new_player2_pos = new_player2_pos if new_player2_pos <= 10 else new_player2_pos % 10
        new_player2_score = player2_score + new_player2_pos
        score_board[player1_pos - 1, new_player2_pos - 1, player1_score, new_player2_score] += n_universes_in_config * n_repetitions
        
    #return output_score_board.copy()

def print_games(df):
    a, b, c, d = np.where(df != 0)
    for i in range(len(a)):
        print(a[i] + 1, b[i] + 1, c[i], d[i], df[a[i], b[i], c[i], d[i]] )
    print()

In [23]:
player1_start_pos, player2_start_pos

(7, 4)

In [24]:
from itertools import product
score_board = np.zeros((10, 10, 31, 31), dtype = np.int64)#pd.DataFrame(data = 0, index = [str(s) for s in list(product(range(1, 11), range(1, 11)))], columns = [str(s) for s in list(product(range(31), range(31)))], dtype = np.int64)
score_board[player1_start_pos - 1, player2_start_pos - 1, 0, 0] = 1
#score_board.loc[str((player1_start_pos, player2_start_pos)), str((0, 0))] = 1
print(score_board[:, :, :21, :21].sum() > 0)

#player1_win_count = 0
#player2_win_count = 0

while score_board[:, :, :21, :21].sum() > 0:
    
    # Player 1 plays
    
    #print_games(input_score_board, output_score_board)
    
    #for positions, scores in running_games.iterrows():#running_games[(running_games != 0).any(axis = 1)].iterrows():
    """player1_positions, player2_positions, player1_scores, player2_scores = np.where(input_score_board!=0)
    for i in range(len(player1_positions)):
        player1_pos = player1_positions[i] + 1 # to get position in [1, 10]
        player2_pos = player2_positions[i] + 1
        player1_score = player1_scores[i]
        player2_score = player2_scores[i]"""
    
    for player1_moving in [True, False]:
        
        input_score_board = score_board.copy()

        for player1_pos in range(1, 11):
            for player2_pos in range(1, 11):
                for player1_score in range(21):
                    for player2_score in range(21):

                        #if player1_score >= 21 or player2_score >= 21: continue
                        
                        a = input_score_board[player1_pos - 1, player2_pos - 1, player1_score, player2_score]

                        create_new_universes(input_score_board, 3, 1, player1_pos, player2_pos, player1_score, player2_score, player1_moving = player1_moving)
                        create_new_universes(input_score_board, 4, 3, player1_pos, player2_pos, player1_score, player2_score, player1_moving = player1_moving)
                        create_new_universes(input_score_board, 5, 6, player1_pos, player2_pos, player1_score, player2_score, player1_moving = player1_moving)
                        create_new_universes(input_score_board, 6, 7, player1_pos, player2_pos, player1_score, player2_score, player1_moving = player1_moving)
                        create_new_universes(input_score_board, 7, 6, player1_pos, player2_pos, player1_score, player2_score, player1_moving = player1_moving)
                        create_new_universes(input_score_board, 8, 3, player1_pos, player2_pos, player1_score, player2_score, player1_moving = player1_moving)
                        create_new_universes(input_score_board, 9, 1, player1_pos, player2_pos, player1_score, player2_score, player1_moving = player1_moving)

                        #if a != score_board[player1_pos - 1, player2_pos - 1, player1_score, player2_score]:
                        #    print("bananas")

                        if input_score_board[player1_pos - 1, player2_pos - 1, player1_score, player2_score] != 0:
                            score_board[player1_pos - 1, player2_pos - 1, player1_score, player2_score] = score_board[player1_pos - 1, player2_pos - 1, player1_score, player2_score] - a  


    #score_board = output_score_board.copy()
 
    print("N universes remaining in game:", score_board[:, :, :21, :21].sum())
    #print("N games player 1 won:", score_board[:, :, 21:, :].sum())
    #print("N games player 2 won:", score_board[:, :, :, 21:].sum())
    #print("N games ended:", score_board[:, :, 21:, :].sum() + score_board[:, :, :, 21:].sum())

True
N universes remaining in game: 729
N universes remaining in game: 531441
N universes remaining in game: 275586075
N universes remaining in game: 41562755877
N universes remaining in game: 1614777104579
N universes remaining in game: 12908082816718
N universes remaining in game: 6973235686560
N universes remaining in game: 121621010764
N universes remaining in game: 4984659
N universes remaining in game: 0


In [25]:
player1_win_count = score_board[:, :, 21:, :].sum()
player2_win_count = score_board[:, :, :, 21:].sum()
player1_win_count, player2_win_count

(570239341223618, 371697814511699)

In [35]:
player1_win_count < 444356092776315

True

In [36]:
player2_win_count < 341960390180808

True

# Day 22

In [149]:
puzzle = Puzzle(year=2021, day=22)
data = lines(puzzle.input_data)

In [150]:
operations = [line[:3].strip() for line in data]
coordinates = [line[3:].strip().split(',') for line in data]
xs = [c[0].replace('x=', '').split('..') for c in coordinates]
x0s = [int(x[0]) for x in xs]
x1s = [int(x[1]) for x in xs]
ys = [c[1].replace('y=', '').split('..') for c in coordinates]
y0s = [int(y[0]) for y in ys]
y1s = [int(y[1]) for y in ys]
zs = [c[2].replace('z=', '').split('..') for c in coordinates]
z0s = [int(z[0]) for z in zs]
z1s = [int(z[1]) for z in zs]

In [151]:
len(x0s) == len(x1s) == len(y0s) == len(y1s) == len(z0s) == len(z1s)

True

In [152]:
len(x0s)

420

In [11]:
%%time
from IPython.display import clear_output

on_coords = set()
for i in range(len(x0s)):
    clear_output()
    print(i)
    for x in range(x0s[i], x1s[i] + 1):
        if x < -50 or x > 50: continue
        for y in range(y0s[i], y1s[i] + 1):
            if y < -50 or y > 50: continue
            for z in range(z0s[i], z1s[i] + 1):
                if z < -50 or z > 50: continue
                if operations[i] == 'on':
                    on_coords.add((x, y, z))
                else:
                    if (x, y, z) in on_coords:
                        on_coords.remove((x, y, z))

419
Wall time: 1min 53s


In [12]:
len(on_coords)

600458

In [153]:
class Cube:
    
    def __init__(self, kind, x0, x1, y0, y1, z0, z1, volume):
        self.kind = kind
        self.x0 = x0
        self.x1 = x1
        self.y0 = y0
        self.y1 = y1
        self.z0 = z0
        self.z1 = z1
        self.volume = self.calculate_volume() if kind == 'on' else -self.calculate_volume()
        
    def display(self):
        print(f"{self.kind} x0 = {self.x0}, x1 = {self.x1}, y0 = {self.y0}, y1 = {self.y1}, z0 = {self.z0}, z1 = {self.z1}, volume = {self.volume}")
        
    def find_overlap_w_cube(self, original_cube):
        overlap_x0 = max(self.x0, original_cube.x0)
        overlap_x1 = min(self.x1, original_cube.x1)
        overlap_y0 = max(self.y0, original_cube.y0)
        overlap_y1 = min(self.y1, original_cube.y1)
        overlap_z0 = max(self.z0, original_cube.z0)
        overlap_z1 = min(self.z1, original_cube.z1)
        if overlap_x1 >= overlap_x0 and overlap_y1 >= overlap_y0 and overlap_z1 >= overlap_z0:
            return overlap_x0, overlap_x1, overlap_y0, overlap_y1, overlap_z0, overlap_z1
        else: return None
    
    def calculate_volume(self):
        return (self.x1 - self.x0 + 1) * (self.y1 - self.y0 + 1) * (self.z1 - self.z0 + 1)
            
class CubeSet:
    
    def __init__(self):
        self.cubes = []
        
    def add_cube(self, new_cube):
        
        if new_cube.kind == 'on':
            cubes_to_add = [new_cube]
        else:
            cubes_to_add = []
            
        for cube in self.cubes:
            intersection = new_cube.find_overlap_w_cube(cube)
            if intersection is None: continue
            if cube.kind == 'on': cubes_to_add.append(Cube('off', *intersection))
            else: cubes_to_add.append(Cube('on', *intersection))
            
        self.cubes += cubes_to_add
        
    def get_n_on(self):
        return sum([cube.volume for cube in self.cubes])

In [157]:
%%time
cube_set = CubeSet()

for i in range(len(x0s)):
    new_cube = Cube(operations[i], x0s[i], x1s[i], y0s[i], y1s[i], z0s[i], z1s[i])
    cube_set.add_cube(new_cube)

Wall time: 7.66 s


In [155]:
cube_set.get_n_on()

1334275219162622

# Day 23

In [3]:
puzzle = Puzzle(year=2021, day=23)
data = lines(puzzle.input_data)

In [4]:
data

['#############',
 '#...........#',
 '###B#B#C#D###',
 '  #D#C#A#A#',
 '  #########']

In [3]:
from random import shuffle

def BFS_SP(graph, start, goal):
    explored = []
     
    # Queue for traversing the
    # graph in the BFS
    queue = [[start]]
     
    # If the desired node is
    # reached
    if start == goal:
        print("Same Node")
        return
     
    # Loop to traverse the graph
    # with the help of the queue
    while queue:
        path = queue.pop(0)
        node = path[-1]
         
        # Condition to check if the
        # current node is not visited
        if node not in explored:
            neighbours = graph[node]
             
            # Loop to iterate over the
            # neighbours of the node
            for neighbour in neighbours:
                new_path = list(path)
                new_path.append(neighbour)
                queue.append(new_path)
                 
                # Condition to check if the
                # neighbour node is the goal
                if neighbour == goal:
                    #print("Shortest path = ", *new_path)
                    return new_path
            explored.append(node)
 
    # Condition when the nodes
    # are not connected
    print("So sorry, but a connecting"\
                "path doesn't exist :(")
    return

class Amphipod:
    
    def __init__(self, letter, init_position):
        self.letter = letter
        self.position = init_position
        self.has_ever_been_outside = False
        self.burrow_positions = []
    
    def move(self, new_position):
        if new_position not in self.burrow_positions:
            if self.has_ever_been_outside: return None
            else:
                self.position = new_position
                self.has_ever_been_outside = True
                return new_position
        self.position = new_position
        return new_position
        
    def is_in_right_burrow(self):
        return self.position in self.burrow_positions
    
    def get_energy_multiplier(self):
        if self.letter == 'A': return 1
        if self.letter == 'B': return 10
        if self.letter == 'C': return 100
        if self.letter == 'D': return 1000
    
    def is_in_final_position(self, state):
        if self.position not in self.burrow_positions: return False
        return all(state[self.position:max(self.burrow_positions) + 1] == self.letter)
    
class Burrows:
    
    def __init__(self, size = 19):
        self.amphipods = []
        self.size = size
        self.burrow_size = int((size - 11) / 4)
        self.spent_energy = 0
        self.burrow_map = {0: [1],
                    1: [0, 2],
                    2: [1, 3, 11],
                    3: [2, 4],
                    4: [3, 5, 11 + self.burrow_size],
                    5: [4, 6],
                    6: [5, 7, 11 + 2*self.burrow_size],
                    7: [6, 8],
                    8: [7, 9, 11 + 3*self.burrow_size],
                    9: [8, 10],
                    10: [9]}

        for i in range(4):
            burrow_top = 11 + i*self.burrow_size
            self.burrow_map[burrow_top] = [[2, 4, 6, 8][i], burrow_top + 1]
            for j in range(1, self.burrow_size - 1):
                self.burrow_map[burrow_top + j] = [burrow_top + j - 1, burrow_top + j + 1]
            self.burrow_map[burrow_top + self.burrow_size - 1] = [burrow_top + self.burrow_size - 2]

        self.state = np.full(shape=(self.size), fill_value = '.')
        
        self.burrow_positions = list(range(11, self.size))
        self.burrow_positions_per_letter = {['A', 'B', 'C', 'D'][i]: list(range(11+i*self.burrow_size, 11+(i+1)*self.burrow_size)) for i in range(4)}
        
    def print_state(self):
        print("#############")
        print(f"#{''.join([self.state[i] for i in range(11)])}#")
        for i in range(self.burrow_size):
            print(f"###{self.state[11 + i]}#{self.state[11 + self.burrow_size + i]}#{self.state[11 + 2*self.burrow_size + i]}#{self.state[11 + 3*self.burrow_size + i]}###")
        print("  #########  ")
    
    def add_amphipod(self, amphipod):
        amphipod.burrow_positions = self.burrow_positions_per_letter[amphipod.letter]
        self.amphipods.append(amphipod)
        self.state[amphipod.position] = amphipod.letter
        
    def find_available_positions(self):
        return [i for i in range(self.size) if i not in [2, 4, 6, 8] + [amphipod.position for amphipod in self.amphipods]]
    
    def is_game_over(self):
        return all([amphipod.is_in_right_burrow() for amphipod in self.amphipods])
    
    def list_allowed_movements(self, previous_moved_amphipod_idx = None):
        allowed_movements = []
        energies = []
        available_positions = self.find_available_positions()
        
        burrow_moves_idx = []
        idx = 0
        for i, amphipod in enumerate(self.amphipods):
            if amphipod.is_in_final_position(self.state): continue
            if previous_moved_amphipod_idx is not None and i == previous_moved_amphipod_idx: continue
            for new_position in available_positions:
                
                # If the new position is in a burrow it must be in the burrow that corresponds to the amphipod
                if new_position in self.burrow_positions and new_position not in self.burrow_positions_per_letter[amphipod.letter]: 
                    #print(amphipod.position, new_position,"Amphipode can only be moved to it's own burrow")
                    continue
                
                # If it's in the right burrow then we should make sure that there is not another different amphipod bellow or we don't leave an empty space
                if new_position in self.burrow_positions:
                    if not all(self.state[new_position + 1:max(self.burrow_positions_per_letter[amphipod.letter]) + 1] == amphipod.letter):
                        #print(amphipod.position, new_position,"Don't want to leave an empty space or amphipod locked in")
                        continue
                
                # If it's outside we need to check that the amphipod can still move there
                if new_position not in self.burrow_positions and amphipod.has_ever_been_outside: 
                    #print(amphipod.position, new_position, "Amphipod cannot be moved twice to outside a burrow")
                    continue
                
                # Check that the path is free
                path = BFS_SP(self.burrow_map, amphipod.position, new_position)
                if any(self.state[path[1:]] != '.'): 
                    #print(amphipod.position, new_position, "There is another amphipod on the way")
                    continue
                
                if new_position in self.burrow_positions_per_letter[amphipod.letter]: 
                    burrow_moves_idx.append(idx)
                
                allowed_movements.append((i, new_position, amphipod.get_energy_multiplier() * (len(path) - 1)))
                #print(allowed_movements[-1])
                
                idx += 1

        #return sorted(allowed_movements, key=lambda pair: pair[2])
        temp = [allowed_movements[i] for i in range(len(allowed_movements)) if i not in burrow_moves_idx]
        shuffle(temp)
        return [allowed_movements[i] for i in burrow_moves_idx] + temp
    
    def move_amphipod(self, movement):
        amphipod_idx, new_position, energy = movement
        old_position = self.amphipods[amphipod_idx].position
        move_position = self.amphipods[amphipod_idx].move(new_position)
        if move_position is None: return None
        self.state[old_position] = '.'
        self.state[new_position] = self.amphipods[amphipod_idx].letter
        self.spent_energy += energy
        return move_position

In [6]:
def setup():
    burrows = Burrows()
    burrows.add_amphipod(Amphipod('B', 11)) #0
    burrows.add_amphipod(Amphipod('D', 12)) #1
    burrows.add_amphipod(Amphipod('B', 13)) #2
    burrows.add_amphipod(Amphipod('C', 14)) #3
    burrows.add_amphipod(Amphipod('C', 15)) #4
    burrows.add_amphipod(Amphipod('A', 16)) #5
    burrows.add_amphipod(Amphipod('D', 17)) #6
    burrows.add_amphipod(Amphipod('A', 18)) #7
    return burrows

In [7]:
game = setup()
game.print_state()

#############
#...........#
###B#B#C#D###
###D#C#A#A###
  #########  


In [8]:
move = (4, 7, 200)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
    
game.print_state()

#############
#.......C...#
###B#B#.#D###
###D#C#A#A###
  #########  


In [153]:
move = (5, 1, 7)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.....C...#
###B#B#.#D###
###D#C#.#A###
  #########  


In [154]:
move = (2, 3, 20)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.B...C...#
###B#.#.#D###
###D#C#.#A###
  #########  


In [155]:
move = (4, 16, 300)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.B.......#
###B#.#.#D###
###D#C#C#A###
  #########  


In [156]:
move = (3, 15, 500)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.B.......#
###B#.#C#D###
###D#.#C#A###
  #########  


In [157]:
move = (2, 14, 30)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.........#
###B#.#C#D###
###D#B#C#A###
  #########  


In [158]:
move = (0, 13, 40)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.........#
###.#B#C#D###
###D#B#C#A###
  #########  


In [159]:
move = (6, 7, 2000)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.....D...#
###.#B#C#.###
###D#B#C#A###
  #########  


In [160]:
move = (7, 9, 3)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.....D.A.#
###.#B#C#.###
###D#B#C#.###
  #########  


In [161]:
move = (6, 18, 3000)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.......A.#
###.#B#C#.###
###D#B#C#D###
  #########  


In [162]:
game.list_allowed_movements()

[(1, 17, 9000), (1, 3, 3000), (1, 5, 5000), (1, 7, 7000)]

In [163]:
move = (1, 17, 9000)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.A.......A.#
###.#B#C#D###
###.#B#C#D###
  #########  


In [164]:
move = (5, 12, 3)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#.........A.#
###.#B#C#D###
###A#B#C#D###
  #########  


In [165]:
move = (7, 11, 8)
if move in game.list_allowed_movements():
    moved_position = game.move_amphipod(move)
else: print("Move not allowed")
game.print_state()

#############
#...........#
###A#B#C#D###
###A#B#C#D###
  #########  


In [166]:
game.is_game_over()

True

In [9]:
from IPython.display import clear_output

best_score = np.inf
initial_game = setup()
move_stack = [[(4, 7, 200)]]#[[move] for move in initial_game.list_allowed_movements()]

#explored_configurations = []

while len(move_stack) > 0:

    stack = move_stack.pop(0)
    
    game = setup()
    for move in stack:
        moved_position = game.move_amphipod(move)

    if game.is_game_over():
        #print("Current best score", best_score)
        if game.spent_energy < best_score: 
            best_score = game.spent_energy
            print("Found a new best", best_score)
        continue
        
    if game.spent_energy > best_score: continue
    #if any([all(game.state == s) for s in explored_configurations]): continue
    else:
        previously_moved_amphipod_idx = stack[-1][0]
        new_allowed_moves = game.list_allowed_movements(previously_moved_amphipod_idx)
        move_stack = [stack + [move] for move in new_allowed_moves] + move_stack
        #explored_configurations.append(game.state)

Found a new best 17147
Found a new best 15251
Found a new best 15191
Found a new best 15151
Found a new best 15133
Found a new best 15113
Found a new best 15111


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [10]:
best_score

15111

## Part 2

In [4]:
def setup():
    burrows = Burrows(27)
    burrows.add_amphipod(Amphipod('B', 11)) #0
    burrows.add_amphipod(Amphipod('D', 12)) #1
    burrows.add_amphipod(Amphipod('D', 13)) #2
    burrows.add_amphipod(Amphipod('D', 14)) #3
    burrows.add_amphipod(Amphipod('B', 15)) #4
    burrows.add_amphipod(Amphipod('C', 16)) #5
    burrows.add_amphipod(Amphipod('B', 17)) #6
    burrows.add_amphipod(Amphipod('C', 18)) #7
    burrows.add_amphipod(Amphipod('C', 19)) #8
    burrows.add_amphipod(Amphipod('B', 20)) #9
    burrows.add_amphipod(Amphipod('A', 21)) #10
    burrows.add_amphipod(Amphipod('A', 22)) #11
    burrows.add_amphipod(Amphipod('D', 23)) #12
    burrows.add_amphipod(Amphipod('A', 24)) #13
    burrows.add_amphipod(Amphipod('C', 25)) #14
    burrows.add_amphipod(Amphipod('A', 26)) #15
    return burrows

In [5]:
setup().print_state()

#############
#...........#
###B#B#C#D###
###D#C#B#A###
###D#B#A#C###
###D#C#A#A###
  #########  


In [6]:
from IPython.display import clear_output

best_score = np.inf
initial_game = setup()
move_stack = [[move] for move in initial_game.list_allowed_movements()]

#explored_configurations = []

while len(move_stack) > 0:

    stack = move_stack.pop(0)
    
    game = setup()
    for move in stack:
        moved_position = game.move_amphipod(move)

    if game.is_game_over():
        #print("Current best score", best_score)
        if game.spent_energy < best_score: 
            best_score = game.spent_energy
            print("Found a new best", best_score)
        continue
        
    if game.spent_energy > best_score: continue
    #if any([all(game.state == s) for s in explored_configurations]): continue
    else:
        previously_moved_amphipod_idx = stack[-1][0]
        new_allowed_moves = game.list_allowed_movements(previously_moved_amphipod_idx)
        move_stack = [stack + [move] for move in new_allowed_moves] + move_stack
        #explored_configurations.append(game.state)

Found a new best 48585
Found a new best 48565
Found a new best 47625


In [7]:
best_score

47625

# Day 24
Solved by hand

# Day 25

In [55]:
puzzle = Puzzle(year=2021, day=25)
data = lines(puzzle.input_data)

In [45]:
data = ['v...>>.vv>',
'.vv>>.vv..',
'>>.>v>...v',
'>>v>>.>.v.',
'v>v.vv.v..',
'>.>>..v...',
'.vv..>.>v.',
'v.v..>>v.v',
'....v..v.>']

In [56]:
data = [[letter for letter in line] for line in data]
data = np.array(data)
cucumber_map = np.zeros(data.shape)
cucumber_map[data == '>'] = 1
cucumber_map[data == 'v'] = 2
cucumber_map = cucumber_map.astype(int)

In [57]:
def move_cucumbers(cucumber_map, direction = 'east'):
    new_cucumber_map = cucumber_map.copy()
    moved_cucumbers = 0
    
    if direction == 'east':
        rows, cols = np.where(cucumber_map == 1)
    else:
        rows, cols = np.where(cucumber_map == 2)
        
    for i, j in zip(rows, cols):
        if direction == 'east':
            next_i = i
            next_j = j + 1 if j + 1 < cucumber_map.shape[1] else 0
        else:
            next_i = i + 1 if i + 1 < cucumber_map.shape[0] else 0
            next_j = j 
            
        # If the next position is free move the cucumbers
        if cucumber_map[next_i, next_j] == 0:
            moved_cucumbers += 1
            new_cucumber_map[i, j] = 0
            new_cucumber_map[next_i, next_j] = 1 if direction == 'east' else 2
            
    return new_cucumber_map, moved_cucumbers

In [58]:
# Take 1 step
new_cucumber_map, moved_east_cucumbers = move_cucumbers(cucumber_map, direction = 'east')
new_cucumber_map, moved_south_cucumbers = move_cucumbers(new_cucumber_map, direction = 'south')

total_moved_cucumbers = moved_east_cucumbers + moved_south_cucumbers

step_count = 1

while total_moved_cucumbers != 0:
    new_cucumber_map, moved_east_cucumbers = move_cucumbers(new_cucumber_map, direction = 'east')
    new_cucumber_map, moved_south_cucumbers = move_cucumbers(new_cucumber_map, direction = 'south')

    total_moved_cucumbers = moved_east_cucumbers + moved_south_cucumbers
    step_count += 1

In [59]:
step_count

424